# Baking PV

Notebook I used to bake PV data into the hdf5 file, before realizing that I don't want to do that and just saved PV data as a pickle instead..
(so this notebook is not used but I saved it anyways)

In [ ]:
from datetime import datetime, time, timedelta
from pathlib import Path
import pandas as pd
import xarray as xr
from ocf_blosc2 import Blosc2
import json
import pickle
import h5py
from tqdm import tqdm
import numpy as np

In [ ]:
pv = pd.concat([
    pd.read_parquet(f"/data/climatehack/official_dataset/pv/{y}/{i}.parquet").drop("generation_wh", axis=1)
    for y in (2020, 2021)
    for i in range(1, 13)
])

In [ ]:
h5file = h5py.File("../data/data.h5", "a")

bake_index_old = h5file['bake_index'][:]

In [ ]:
bake_index_dt = np.dtype([
    ('time', np.int32),
    ('site', np.int32),
    ('nonhrv_flags', np.bool_, (11,)),
    ('weather_flags', np.bool_, (38,)),
    ('pv_features', np.float32, (12,)),
    ('pv_target', np.float32, (48,)),
], align=True)
bake_index = np.zeros_like(bake_index_old, dtype=bake_index_dt)

In [ ]:
((bake_index_old['time'][1:] - bake_index_old['time'][:-1]) >= 0).all()

In [ ]:
old_ts = -1
pv_features = None
pv_targets = None
for i, row in enumerate(tqdm(bake_index_old)):
    if old_ts != row['time']:
        old_ts = row['time']
        time = datetime.fromtimestamp(row['time'])
        pv_features = pv.xs(
            slice(
                str(time),
                str(time + timedelta(minutes=55))
            ),
        )  # type: ignore
        pv_targets = pv.xs(
            slice(  # type: ignore
                str(time + timedelta(hours=1)),
                str(time + timedelta(hours=4, minutes=55)),
            ),
        )
    site_features = pv_features.xs(row['site']).to_numpy().squeeze(-1)
    site_targets = pv_targets.xs(row['site']).to_numpy().squeeze(-1)
    bake_index[i] = (
        row['time'],
        row['site'],
        row['nonhrv_flags'],
        row['weather_flags'],
        site_features,
        site_targets
    )

In [ ]:
del h5file['bake_index']
ds = h5file.create_dataset(
        'bake_index',
        shape=(len(bake_index),),
        dtype=bake_index_dt,
        chunks=(min(10000, len(bake_index)),),
)
ds[:] = bake_index

In [ ]:
h5file.close()